# Working with MySQL from Python
In this tutorial you are shown how to execute SQL statements on a MySQL database from a Python script.

It is assumed here that you have completed Lab 3 and have a basic understanding of relational models and SQL.

**Please note that, since this is a notebook file, you cannot run it from terminal like a .py script.**

## MySQL Connector API
In this tutorial we will be using the [MySQL Connector API](https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html) for Python. This library provides a set of utilities we can use to connect to and query a MySQL database. API stands for \`Application Programming Interface', and it is a general term used to describe a set of classes/functions/routines that facilitate the interaction between 2 different software applications (or different components in the same application).


In [1]:
# import the mysql connector API
import mysql.connector

In [44]:
# store some reuseable configuration options
# edit these according to your database settings
config = {
    'user' : 'USERNAME',
    'password' : 'PASSWORD',
    'host' : 'localhost',
    'database' : 'DATABASENAME'
}

## Connect to a database
First off, we will define a reuseable function that establishes a connection with a database, according to the configuration options that we pass it. We will also use this function to try to handle and report on any connection errors.

In [45]:
def connect(config):
    """ Creates a connection with a MySQL database
        Returns a connection object (handle to the database)
    """

    try:
        cnx = mysql.connector.connect(**config)
        print( "Connected to {} database as {}".format( config['database'], config['user'] ) )
        return cnx

    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print( "Something is wrong with your user name or password" )
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print( "Database does not exist" )
        else:
            print(err)

    else:
        cnx.close()


## Query the database
Next we're going to do some work on the database. For that, we need to establish a connection with the database, and then instantiate a **cursor** object. A cursor is a control structure that enables traversal over the records in the database.

In [49]:
# create a connection handle to database
cnx = connect(config)

# check there's an open connection
if cnx:
    
    # create a cursor object
    cursor = cnx.cursor()
    
    print("\nQuerying database...")

    # fetch some data (i.e. the number of times each image has been `served')
    query = ("SELECT m.id AS media_id, m.img_url, COUNT(*) AS num_serves FROM Media m INNER JOIN MediaServe ms ON ms.media_id=m.id GROUP BY media_id")
    cursor.execute(query)
    
    # display some column headings
    print("\nImg id\tImg URL\t\t\t\t\t\tFlucks given")
    
    # display the data returned in the cursor object
    for (media_id, img_url, num_serves) in cursor:
        print("{}\t{}\t{}".format( media_id, img_url, num_serves ))


Connected to flucks database as sorrel

Querying database...

Img id	Img URL						Flucks given
1	https://pbs.twimg.com/media/DMvdd6gWkAAehPL.jpg	7
2	https://pbs.twimg.com/media/DMvfNuCXUAAW4F1.jpg	9
3	https://pbs.twimg.com/media/DMvE0hsXUAAs4pl.jpg	4
4	https://pbs.twimg.com/media/DMvXMmRU8AACFWi.jpg	2
5	https://pbs.twimg.com/media/DMvQbNAXcAAyTI8.jpg	6
6	https://pbs.twimg.com/media/DMviSSOVoAAZyy5.jpg	4
7	https://pbs.twimg.com/media/DMr4y9nXkAAWbB4.jpg	6
8	https://pbs.twimg.com/media/DMvWP1PWsAA6lPP.jpg	7
9	https://pbs.twimg.com/media/DE6JC4AU0AEFE6k.jpg	6
10	https://pbs.twimg.com/media/DMvdFlhW0AA4x-S.jpg	8
11	https://pbs.twimg.com/media/DMr2cTxVQAAneAm.jpg	6
12	https://pbs.twimg.com/media/DMvVQk5W4AAI5Ss.jpg	4
13	https://pbs.twimg.com/media/DMve8MtX0AEzX87.jpg	5
14	https://pbs.twimg.com/media/DMvm75YWAAAsCDL.jpg	7
15	https://pbs.twimg.com/media/DMu6hT2WAAAjIdq.jpg	3
16	https://pbs.twimg.com/media/DMvW43jUIAAwsar.jpg	6
17	https://pbs.twimg.com/media/DMnrbtNU8AEC6lY.jpg	3
18	https://pb

## Inserting data
We can follow a similar process when inserting data. The only difference here is that the data to be inserted is being passed as a second argument in the execute method of the cursor object. This is an optional variation on what we saw previously, where the entire SQL string was passed as a single argument. 

The advantage of passing the data separately for an insert is that it leverages Python's string formating rules to run checks on the data that is being inserted (i.e. does the value match the type indicated by the type **tokens**). Of course, that's assuming that the right tokens are used!

We can also retrieve the **lastrowid** property from the cursor - useful when a single insert is part of **transaction**, in which a subsequent insert updates a child table.

In [51]:
# check there's an open connection
if cnx:
    
    # define the SQL statement (inserts new row in Media)
    # note the use of placeholder `token' 
    # which helps to ensure the integrity of the data
    sql = ("INSERT INTO Media "
                   "(img_url,alt_txt) "
                   "VALUES (%s,%s)")
    
    # define values to replace the tokens
    data = ("https://pbs.twimg.com/media/DM0o8UwXUAAAFm2.jpg","Image related to 'kitten' scraped from Twitter")

    # execute the query on the database
    cursor.execute(sql, data)
    
    # get id of the new row
    fluck_id = cursor.lastrowid
    print(fluck_id)

    # make sure data is committed to the database
    cnx.commit() 

21


## Close the connection
Finally, we should remember to close the cursor and connection with the database. This is because, if many connections are left open, it might affect the performance of the queries. In practice, leaving connections open (or establishing multiple connections) is unlikely to have a noticeable effect on performance until many users are running the application, but it is not a good habit to get into!

In [53]:
# when we finish working, close the connection
cursor.close()
cnx.close()